<a href="https://colab.research.google.com/github/ahmedmusawir/modern-agents-youtube-series/blob/main/New_React_Agent_w_Custom_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
!pip install -q langchain
!pip install -q langchain_openai
!pip install -q langchain_community
!pip install -q langchain-experimental
!pip install -q langgraph
!pip install -q duckduckgo-search
!pip install -q wikipedia

!pip show langchain

Name: langchain
Version: 0.3.2
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


## OpenAI Key Setup

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


## Model Testing

In [ ]:
from langchain_openai import ChatOpenAI
import pprint
import logging

# Set logging level to ERROR to suppress warnings and info logs
logging.getLogger("langchain_core").setLevel(logging.ERROR)

# Create an instance of the OpenAI class
llm = ChatOpenAI(model="gpt-4o")

# Generate text
prompt = "Who is Trump (in one sentence)"
response = llm.invoke(prompt)
# print(response)
pprint.pprint(response.content)

('Donald Trump is an American businessman, television personality, and '
 'politician who served as the 45th President of the United States from 2017 '
 'to 2021.')


# NEW RE-ACT AGENT

## THE TOOLS

In [ ]:
# Import the required modules
import wikipedia
import pprint
from langchain_core.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Define the tool for DuckDuckGo search
@tool
def search_duckduckgo(query: str) -> str:
    """Search DuckDuckGo for a given query and return the first result snippet."""
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=1)  # Customize max_results as needed
    # results = wrapper.results(query)
    results = wrapper.results(query=query, max_results=1)
    if results:
        result = results[0]  # Get the first result
        return f"Title: {result['title']}\nSnippet: {result['snippet']}\nLink: {result['link']}"
    return "No results found."

# Successfully tested the tool
# duck_result = search_duckduckgo.invoke("Donald Trump")
# pprint.pprint(duck_result)

# Import the required modules for Wikipedia search
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

# Define the Wikipedia search tool
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for a given query and return a summary."""
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.run(query)

# Successfully tested the tool
wiki_result = search_wikipedia.invoke("Donald Trump")
print("WIKI RESULT", wiki_result)




## CUSTOM VERBOSE UTILITY

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
# from pprint import pprint

def parse_agent_messages(messages):
    for msg in messages:
        if isinstance(msg, HumanMessage):
            print(f"User: {msg.content}\n")
        elif isinstance(msg, AIMessage):
            if 'tool_calls' in msg.additional_kwargs and msg.additional_kwargs['tool_calls']:
                print("Agent is deciding to use tools...\n")
                for tool_call in msg.additional_kwargs['tool_calls']:
                    tool_name = tool_call['function']['name']
                    arguments = tool_call['function']['arguments']
                    print(f"Agent calls tool: {tool_name} with arguments {arguments}\n")
            else:
                print(f"Agent's Final Response:\n{msg.content}\n")
        elif isinstance(msg, ToolMessage):
            tool_name = msg.name
            print(f"Tool [{tool_name}] Response:\n{msg.content}\n")
        else:
            print(f"Unknown message type: {msg}\n")

## PRE-BUILT REACT AGENT FROM LANGGRAPH

In [ ]:
# Import LangGraph prebuilt utilities
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import pprint

# Set up the language model
model = ChatOpenAI(model="gpt-4o", temperature=0.5)

# List of tools we created
tools = [search_duckduckgo, search_wikipedia]

# Create a LangGraph agent using the tools
new_react_agent = create_react_agent(model, tools)

# Define a simple user query to invoke the agent
query = "tell me about U.S. Election 2024"

# Invoke the agent with the query wrnew_react_agented in a message
messages = new_react_agent.invoke({"messages": [("human", query)]})

# Print the final output
# pprint.pprint(messages["messages"])

# pprint.pprint({
#     "input": query,
#     "output": messages["messages"][-1].content,
# })

output = messages["messages"][-1].content
print(output)

# Use the function on your messages
# parse_agent_messages(messages["messages"])


The 2024 United States elections are scheduled for Tuesday, November 5, 2024. Here's a detailed overview of the elections:

### Presidential Election
- **Date**: November 5, 2024
- **Nature**: 60th quadrennial presidential election.
- **Process**: Voters in each state and the District of Columbia will choose electors for the Electoral College, who will then elect the president and vice president.
- **Democratic Party**: Initially, incumbent President Joe Biden ran for re-election but withdrew on July 21, endorsing Vice President Kamala Harris, who became the party's nominee on August 5. Harris chose Tim Walz, Governor of Minnesota, as her running mate.
- **Republican Party**: Former President Donald Trump is running for a second, non-consecutive term. He was nominated during the 2024 Republican National Convention with Ohio Senator JD Vance as his running mate. Trump has been involved in controversies, including false claims about the 2020 election and legal issues.

### Congressional 